##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pose Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect pose landmarks from images.

## Preparation

Let's start with installing MediaPipe.


In [2]:
!pip install -q mediapipe

Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models) for more information about this model bundle.

In [3]:
#!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

## Visualization utilities

In [4]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## Download test image

To demonstrate the Pose Landmarker API, you can download a sample image using the follow code. The image is from [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/).

In [5]:
#!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

import cv2

# 이미지 다운로드 (wget 대신 requests 사용)
import requests

#url = "https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg"
#with open("imageByCG.png", "wb") as f:
#    f.write(requests.get(url).content)

# 이미지 읽기 및 표시
img = cv2.imread("pose_demo_3p.png")
cv2.imshow("Image", img)
cv2.waitKey(0)  # 키 입력 대기
cv2.destroyAllWindows()  # 창 닫기

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [6]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## Running inference and visualizing the results

The final step is to run pose landmark detection on your selected image. This involves creating your PoseLandmarker object, loading your image, running detection, and finally, the optional step of displaying the image with visualizations.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python) to learn more about configuration options that this solution supports.


In [7]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
  base_options=base_options,
  num_poses=4,
  output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("pose_demo_3p.png")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)
print(detection_result)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("Pose Landmarks", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


PoseLandmarkerResult(pose_landmarks=[[NormalizedLandmark(x=0.8177568912506104, y=0.28550517559051514, z=-0.440613329410553, visibility=0.999977707862854, presence=0.9999842643737793), NormalizedLandmark(x=0.8277736306190491, y=0.27561649680137634, z=-0.4085700213909149, visibility=0.9999293088912964, presence=0.9999791383743286), NormalizedLandmark(x=0.8323347568511963, y=0.2760326564311981, z=-0.4090164005756378, visibility=0.999932050704956, presence=0.9999843835830688), NormalizedLandmark(x=0.8366383910179138, y=0.2763778567314148, z=-0.40883368253707886, visibility=0.9999493360519409, presence=0.9999818801879883), NormalizedLandmark(x=0.8099424242973328, y=0.2761300206184387, z=-0.41323956847190857, visibility=0.9999266862869263, presence=0.9999593496322632), NormalizedLandmark(x=0.8033714890480042, y=0.27695637941360474, z=-0.41375797986984253, visibility=0.9999347925186157, presence=0.9999648332595825), NormalizedLandmark(x=0.7979804277420044, y=0.27770790457725525, z=-0.41401353

In [8]:
def draw_world_landmarks_with_coordinates(rgb_image, detection_result):
  # 2D 정규화 랜드마크 리스트 (list of list)
  pixel_landmarks_list = detection_result.pose_landmarks
  # 3D 월드 랜드마크 리스트 (list of list)
  world_landmarks_list = detection_result.pose_world_landmarks

  annotated_image = np.copy(rgb_image)
  h, w = annotated_image.shape[:2]

  # 둘 중 하나라도 없으면 원본 반환
  if not pixel_landmarks_list or not world_landmarks_list:
    return annotated_image

  # 인물(사람)별로 순회
  for idx in range(len(pixel_landmarks_list)):
    lm2d_list = pixel_landmarks_list[idx]
    lm3d_list = world_landmarks_list[idx]
    if not lm2d_list or not lm3d_list:
      continue

    # 2D overlay: 연결선 + 점
    pose2d_proto = landmark_pb2.NormalizedLandmarkList()
    pose2d_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=lm.x, y=lm.y, z=lm.z)
      for lm in lm2d_list
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose2d_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style()
    )

    # 3D 월드 좌표 텍스트 표시
    for lm2d, lm3d in zip(lm2d_list, lm3d_list):
      x_px = int(lm2d.x * w)
      y_px = int(lm2d.y * h)
      coord_text = f"({lm3d.x:.2f}m, {lm3d.y:.2f}m, {lm3d.z:.2f}m)"
      # 검은 테두리
      cv2.putText(annotated_image, coord_text, (x_px + 5, y_px - 5),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)
      # 흰 글씨
      cv2.putText(annotated_image, coord_text, (x_px + 5, y_px - 5),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

  return annotated_image


# 사용 예
annotated = draw_world_landmarks_with_coordinates(image.numpy_view(), detection_result)
cv2.imshow("Pose WorldLandmarks with Coordinates", cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


Visualize the pose segmentation mask.

In [9]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2.imshow("Segmentation Mask", visualized_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
